<h1 style="text-align:center;"> DFS event prediction of colon cancer using Binary Genetic Algorithm </h1>

---

<h1 style="text-align:center;">Reading Data</h1>
<hr style="opacity: 0.3">

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("./data/Merged Colorectal Cancer Patient Data.csv", sep=",")
df.head(2)

,ID_REF,Age (in years),Dukes Stage,Gender,Location,DFS (in months),DFS event,Adj_Radio,Adj_Chem,117_at,...,1554911_at,1554912_at,1554914_at,1554915_a_at,1554916_a_at,1554918_a_at,1554919_s_at,1554920_at,213418_at,1554921_a_at
0,GSM877126,62.0,A,Male,Left,108.0,0.0,1.0,0.0,6.029834,...,3.504809,2.856109,3.130712,5.603126,3.091348,4.009692,4.35040,2.512026,7.494499,4.030978
1,GSM877127,77.0,B,Male,Left,40.0,1.0,1.0,0.0,6.344321,...,3.957833,2.988643,3.484177,6.601744,3.141737,4.679732,4.50876,2.427697,8.156995,2.827090


----

<h1 style="text-align:center"> Preprocessing the data (Filter stage) </h1>

## Label Encoding

In [3]:
from sklearn.preprocessing import LabelEncoder
df.head(1)

,ID_REF,Age (in years),Dukes Stage,Gender,Location,DFS (in months),DFS event,Adj_Radio,Adj_Chem,117_at,...,1554911_at,1554912_at,1554914_at,1554915_a_at,1554916_a_at,1554918_a_at,1554919_s_at,1554920_at,213418_at,1554921_a_at
0,GSM877126,62.0,A,Male,Left,108.0,0.0,1.0,0.0,6.029834,...,3.504809,2.856109,3.130712,5.603126,3.091348,4.009692,4.3504,2.512026,7.494499,4.030978


In [4]:
Label_Encoders = {
    "Dukes Stage":None,
    "Gender": None,
    "Location": None
}
# use encoders[i].inverse_transform([data]) to get labels back
for label in Label_Encoders.keys():
    Label_Encoders[label] = LabelEncoder()
    transformed = Label_Encoders[label].fit_transform(df[label])
    df[label] = transformed

df.head(1)

,ID_REF,Age (in years),Dukes Stage,Gender,Location,DFS (in months),DFS event,Adj_Radio,Adj_Chem,117_at,...,1554911_at,1554912_at,1554914_at,1554915_a_at,1554916_a_at,1554918_a_at,1554919_s_at,1554920_at,213418_at,1554921_a_at
0,GSM877126,62.0,0,1,1,108.0,0.0,1.0,0.0,6.029834,...,3.504809,2.856109,3.130712,5.603126,3.091348,4.009692,4.3504,2.512026,7.494499,4.030978


In [5]:
Label_Encoders['Location'].inverse_transform([1])

array(['Left'], dtype=object)

<hr style="opacity: 0.5">

# Changing the type of Age, DFS, ADJ's from float to int

In [6]:
labels = [
    "Age (in years)",
    "DFS (in months)",
    "DFS event",
    "Adj_Radio",
    "Adj_Chem",
]

for label in labels :
    df[label] = df[label].astype(int)
df

,ID_REF,Age (in years),Dukes Stage,Gender,Location,DFS (in months),DFS event,Adj_Radio,Adj_Chem,117_at,...,1554911_at,1554912_at,1554914_at,1554915_a_at,1554916_a_at,1554918_a_at,1554919_s_at,1554920_at,213418_at,1554921_a_at
0,GSM877126,62,0,1,1,108,0,1,0,6.029834,...,3.504809,2.856109,3.130712,5.603126,3.091348,4.009692,4.350400,2.512026,7.494499,4.030978
1,GSM877127,77,1,1,1,40,1,1,0,6.344321,...,3.957833,2.988643,3.484177,6.601744,3.141737,4.679732,4.508760,2.427697,8.156995,2.827090
2,GSM877128,66,2,0,1,49,0,1,0,5.023127,...,3.749518,2.610840,3.406207,6.003205,3.469987,4.781843,4.435054,2.531300,6.058687,2.930454
3,GSM877129,72,3,0,1,45,0,1,1,4.612727,...,3.610429,2.728274,3.534871,5.858326,3.372291,3.653575,5.014994,2.608755,5.488072,2.949848
4,GSM877130,75,2,1,1,40,0,0,1,5.865944,...,3.520434,2.664744,3.314746,5.388115,3.322147,4.666360,5.013006,2.657130,7.410725,3.054697
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,GSM877184,54,2,1,1,22,1,0,0,4.835558,...,3.637413,2.787781,3.519436,6.026848,3.404074,4.103411,4.303329,2.422700,6.072618,2.853109
58,GSM877185,48,3,1,3,64,0,1,0,5.694010,...,3.733335,3.473473,3.237316,6.167214,3.452096,4.724017,4.470426,2.634358,7.175365,2.667533
59,GSM877186,65,2,1,1,25,1,1,0,8.661629,...,3.436002,2.706722,3.349033,6.238026,3.109042,6.897634,3.797626,2.756086,10.121831,2.971542
60,GSM877187,71,0,1,2,58,1,1,0,5.702601,...,3.438260,2.539084,3.334590,5.842051,3.496104,4.883845,4.661893,2.581452,7.232090,2.719928


<hr style="opacity: 0.5">

## Initalizing Training Data 

In [ ]:
Y = df['DFS event']
Y

In [8]:
drop_labels = [
   "DFS event",
   "ID_REF"
]
X = df.drop(drop_labels, axis = 1)
x_labels = X.columns
X.head(2)

,Age (in years),Dukes Stage,Gender,Location,DFS (in months),Adj_Radio,Adj_Chem,117_at,1007_s_at,1053_at,...,1554911_at,1554912_at,1554914_at,1554915_a_at,1554916_a_at,1554918_a_at,1554919_s_at,1554920_at,213418_at,1554921_a_at
0,62,0,1,1,108,1,0,6.029834,9.468893,7.187090,...,3.504809,2.856109,3.130712,5.603126,3.091348,4.009692,4.35040,2.512026,7.494499,4.030978
1,77,1,1,1,40,1,0,6.344321,9.994204,7.705626,...,3.957833,2.988643,3.484177,6.601744,3.141737,4.679732,4.50876,2.427697,8.156995,2.827090


<hr style="opacity: 0.5">

## Pre-processing by CMIM

In [9]:
from CMIM import cmim
# Conditional Mutual Information Maximization

In [10]:
# If the progress bar is not working properly.
# pip install ipywidgets
# jupyter nbextension enable --py widgetsnbextension

In [40]:
#---------- Testing ------------------------
# x_data = [
#     "Dukes Stage",
#     "Gender",
#     "Location", 
#     "117_at"
# ]
# X = df[x_data]
# x_labels = X.columns
# Y = df["DFS event"]
#--------------------------------------------
# CMIM is a custom function which takes two numpy arrays as arguments.
# It gives the mutual_information between the predictor and determine variable.
features_indices, objective_value, mutual_information = cmim(X.to_numpy(), Y.to_numpy(), n_selected_features=200)
features_indices,mutual_information

 10%|█         | 200/1942 [04:17<37:21,  1.29s/it]


(array([  7,   5,   6,   0,   1,   3,   4,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
         27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
         40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
         53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
         66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
         79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
         92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
        105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
        118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
        131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
        144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
        157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
        170, 171, 172, 173, 174, 175, 176, 177, 178

<hr style="opacity: 0.5">

## Selection of top features from candidates

In [12]:
def get_subset_indices(features_index, mutual_information, threshold=0.53):
    return [idx for idx, mutual_info in zip(features_index,mutual_information) if mutual_info > abs(threshold)]

subset_indices =  get_subset_indices(
    features_index=features_indices, 
    mutual_information=mutual_information,
    threshold=0.91
)

In [13]:
# subset of the data
sub_set = pd.DataFrame()
for i in subset_indices:
    label = x_labels[i]
    sub_set[label] = df[label]
sub_set.head(2)
    

C:\Users\seshu\AppData\Local\Temp/ipykernel_112/706108600.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  sub_set[label] = df[label]


,117_at,1007_s_at,1053_at,121_at,1255_g_at,1294_at,1316_at,1320_at,1405_i_at,1431_at,...,1554912_at,1554914_at,1554915_a_at,1554916_a_at,1554918_a_at,1554919_s_at,1554920_at,213418_at,1554921_a_at,1553645_at
0,6.029834,9.468893,7.187090,6.807582,3.084591,7.397013,6.525426,3.848253,7.572331,3.74820,...,2.856109,3.130712,5.603126,3.091348,4.009692,4.35040,2.512026,7.494499,4.030978,2.593240
1,6.344321,9.994204,7.705626,6.893130,3.118704,7.163797,6.391195,4.202085,8.295322,3.11434,...,2.988643,3.484177,6.601744,3.141737,4.679732,4.50876,2.427697,8.156995,2.827090,2.871277


----

<h1 style="text-align:center">  Binary Genetic Algorithm (Wrapper stage) </h1>

In [14]:
## -------------------- Testing ------------------------
# import random 
# test_gene = [random.randint(0,1) for _ in range(1910)]
# test_gene
## -----------------------------------------------------

In [16]:
def naive_bias(x_data, y_data):
    '''
    x_data, y_data must all be in numbers they must be clean
    returns accuracy of naive_bias 
    '''


    from sklearn.model_selection import train_test_split
    X_train,X_test,Y_train,Y_test = train_test_split(x_data,y_data,test_size = 0.3)

    # print(X_train.head(1), Y_train.head(1), Y_test.head(1), X_test.head(1))

    from sklearn.naive_bayes import GaussianNB
    model = GaussianNB()

    model.fit(X_train, Y_train)

    return model.score(X_test, Y_test)

In [15]:
def get_data_from_gene(binary_gene):
    drop_labels = []
    for idx, val in enumerate(binary_gene):
        if not val :
            label = x_labels[idx]
            drop_labels.append(label)
    return X.drop(drop_labels, axis="columns")

In [33]:
## ---------------------- Test --------------------------------
# import logging
# logging.basicConfig(filename="Scores.log", level=logging.DEBUG)
## -------------------------------------------------------------

In [36]:
    # Fitness function = NB(x) * w + (β / mod(α - β))
    # alpha = no_of_features selected by the BGA
    # beta = total no of features
    # NB(x) = Accuracy of Naive Bias calculated using the data given by gene
    # w = constant 
    # α =  total no of available features
    # β =  no of selected features by binary algorithm
W = 0.1
def fitness(binary_gene):
        β = sum(binary_gene) # This gives us the number of selected features
        # print(β)
        α = len(binary_gene) # This gives us the total number of available features
        

        feed_data = get_data_from_gene(binary_gene)
        if feed_data is None or β == 0:
            # print("feed data is None")
            return 0
        acc = naive_bias(feed_data, Y)

        # preventing dividing by zero error
        if not β == α:
            score = acc*W + (β/abs(β-α))
        else:
            score = acc*W
        
        # logging.info(f"{score=}, {β=}, Ratio : {β/abs(β-α)}")
        # if score > 1:
        #     logging.warning("Score is greater than it should be")
        return score

In [18]:
from bga import BGA

imp


In [37]:
num_pop = 20
problem_dimentions = X.shape[1]

test = BGA(pop_shape=(num_pop, problem_dimentions), method=fitness, p_c=0.8, p_m=0.2, max_round = 1000, early_stop_rounds=None, verbose = None, maximum=True)
best_solution, best_fitness = test.run()


100%|██████████| 1000/1000 [15:58<00:00,  1.04it/s]


 Solution: [1 1 0 ... 0 1 0] 
 Fitness: 1.2732976509069283 
 Evaluation times: 20020


----

<h1 style="text-align:center"> Crossfold validation of different algorithms </h1>

In [38]:
best_solution

array([1, 1, 0, ..., 0, 1, 0])

In [39]:
data_frame = get_data_from_gene(best_solution)
data_frame

,Age (in years),Dukes Stage,Location,DFS (in months),1053_at,121_at,1294_at,1405_i_at,1431_at,1494_f_at,...,1554903_at,1554905_x_at,1554908_at,1554910_at,1554912_at,1554915_a_at,1554916_a_at,1554918_a_at,1554919_s_at,213418_at
0,62,0,1,108,7.187090,6.807582,7.397013,7.572331,3.748200,4.311960,...,5.499428,5.451683,2.886594,3.019915,2.856109,5.603126,3.091348,4.009692,4.350400,7.494499
1,77,1,1,40,7.705626,6.893130,7.163797,8.295322,3.114340,4.257666,...,6.721471,6.797079,2.888321,3.090021,2.988643,6.601744,3.141737,4.679732,4.508760,8.156995
2,66,2,1,49,7.934057,7.165602,7.375224,7.055579,4.172270,4.390612,...,5.581068,5.623927,2.718859,3.009814,2.610840,6.003205,3.469987,4.781843,4.435054,6.058687
3,72,3,1,45,7.446374,7.071417,7.596233,9.683806,3.625413,4.308245,...,5.624770,5.618488,2.846498,2.657571,2.728274,5.858326,3.372291,3.653575,5.014994,5.488072
4,75,2,1,40,7.141105,7.103383,7.278244,7.360892,3.338311,4.188543,...,5.870199,5.942009,3.074338,3.112369,2.664744,5.388115,3.322147,4.666360,5.013006,7.410725
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,54,2,1,22,7.471584,7.920227,7.628249,7.830382,3.161023,4.534604,...,5.411332,5.515978,2.825752,3.045702,2.787781,6.026848,3.404074,4.103411,4.303329,6.072618
58,48,3,3,64,7.488735,6.699796,7.192814,7.271917,3.392471,4.073181,...,5.363828,5.409951,2.592553,2.919048,3.473473,6.167214,3.452096,4.724017,4.470426,7.175365
59,65,2,1,25,7.512632,7.046389,5.992824,5.335867,3.381974,4.044226,...,6.016838,6.279162,2.827190,3.264191,2.706722,6.238026,3.109042,6.897634,3.797626,10.121831
60,71,0,2,58,7.476872,7.349701,6.878941,8.280592,3.222998,4.674289,...,5.373235,5.634053,2.567928,2.982875,2.539084,5.842051,3.496104,4.883845,4.661893,7.232090


In [22]:
## ---------------------------- Test ---------------------------------
# import matplotlib.pyplot as plt 
# for label in data_frame:
#     data_frame['Gender'].value_counts().sort_index().plot(kind='bar')
#     plt.title(label)
#     plt.grid()
#     plt.show()
## --------------------------------------------------------------------

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

In [24]:
def svm(X, Y):
    X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size = 0.3)
    model = SVC()
    model.fit(X_train, Y_train)

    return model.score(X_test, Y_test)

def knn(X, Y):
    X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size = 0.3)
    model = KNeighborsClassifier()
    model.fit(X_train, Y_train)

    return model.score(X_test, Y_test)

def decision_tree(X, Y):
    X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size = 0.3)
    model = DecisionTreeClassifier()
    model.fit(X_train, Y_train)

    return model.score(X_test, Y_test)

In [25]:
models = [
    svm, 
    knn, 
    decision_tree,
]
acc = []
for model in models:
    acc.append(model(data_frame,Y))

print(acc)

[0.5263157894736842, 0.47368421052631576, 0.5263157894736842]


In [26]:
import pickle
def save_gene(gene, name):
    with open(f"./saves/{name}.pkl", "wb") as f:
        pickle.dump(gene, f)